# Using OSMnx to Collect and Manipulate Transport Networks

## Learning Objectives
By the end of this practical lab you will be able to:

* Create street networks using the OSMnx package
* Extract street networks for different modes
* Run some basic network statistics
* Export your street networks for use in ArcGIS
* Create figure-ground images

## Important - Before You Begin
**OSMnx requires the *geopandas* package**  
Make sure you have downloaded and installed the packages below separately! See the notes in the **P2_Data_Manipulation** notebook. If you did not install these, or want to check if they are installed, you can **run this chunk of code:**

In [ ]:
# Replace YOUR_USERNAME with with your username on the PC
# If you synced the GitHub repository to a different location, you will have to change the code to point to it
!pip install C:\Users\YOUR_USERNAME\Documents\GitHub\LSGI545_2019\00_setup\GDAL-2.3.3-cp37-cp37m-win_amd64.whl
!pip install C:\Users\YOUR_USERNAME\Documents\GitHub\LSGI545_2019\00_setup\Fiona-1.8.4-cp37-cp37m-win_amd64.whl
!pip install C:\Users\YOUR_USERNAME\Documents\GitHub\LSGI545_2019\00_setup\pyproj-1.9.6-cp37-cp37m-win_amd64.whl
!pip install C:\Users\YOUR_USERNAME\Documents\GitHub\LSGI545_2019\00_setup\Rtree-0.8.3-cp37-cp37m-win_amd64.whl
!pip install C:\Users\YOUR_USERNAME\Documents\GitHub\LSGI545_2019\00_setup\Shapely-1.6.4.post1-cp37-cp37m-win_amd64.whl
!pip install descartes
!pip install geopandas

## Downloading and Constructing Networks
OSMnx is a python package that lets you download street networks from OpenStreetMap and project, visualize and save them as shapefiles for use in ArcGIS and other applications. OSMnx has several methods for downloading and constructing networks from OpenStreetMap data. You can find all the methods in the overview tutorial at [OSMnx Tutorials](https://github.com/gboeing/osmnx-examples/tree/master/notebooks) in the [OSMnx Repository](https://github.com/gboeing/osmnx-examples). Let's look at two of them:

In [ ]:
# Download and install the OSMnx package
# You only need to run the following lines the first time you run this notebook
!pip install osmnx
!pip install networkx==2.2

In [ ]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import numpy as np
import requests
%matplotlib inline
ox.config(log_file=True, log_console=True, use_cache=True)

### Network Types
The default network type considers all types of links or edges from OpenStreetMap. However, OSMnx supports downloading several types of networks based on the OpenStreetMap data by specifying a network_type string parameter:
* Driving (*network_type = 'drive'*)
* Driving and Service Roads (*network_type = 'drive_service'*)
* Walking (*network_type = 'walk'*)
* Cycling (*network_type = 'bike'*)

### Method 1: Network from a Point
Constructs the network *G1* from all the OSM nodes and edges around some point:

In [ ]:
# Define a point at some location based on its Latitude and Longitude
location_point = (22.306436, 114.179426)

# create network from point, inside bounding box of N, S, E, W each 1000m from point
G1 = ox.graph_from_point(location_point, distance=1000, distance_type='bbox', network_type='drive')

# Project the network graph into UTM coordinates so they can be used in metres instead of lat-long
G1_projected = ox.project_graph(G1)

# Plot the projected network
fig, ax = ox.plot_graph(G1_projected, node_size=25, node_color='#66cc66')

### Method 2: Network from a Place name
This geocodes the place name, gets the place's boundary shape polygon and bounding box, downloads the network within the bounding box, then prunes all nodes that lie outside the place's boundary polygon and creates the network. Let's get a walking network that we will call *G2*:

In [ ]:
# create the street network within the boundary of Hong Kong
G2 = ox.graph_from_place('Hong Kong, Hong Kong', network_type='drive_service')

# Project the network into UTM coordinates (this can take a while with large networks!)
G2_projected = ox.project_graph(G2)

# Plot the projected network
fig, ax = ox.plot_graph(G2_projected)

### Method 3: Network from Multiple Place Names
You might find that the place name you entered above does not cover the whole area you look to study. This can happen when looking for networks that break due to city boundaries. In this case, the OSMnx tool can handle multiple place names to create network *G3*:

In [ ]:
# You can pass multiple places as a mixed list of strings and/or structured queries
places = ['Manhattan, New York City', 'The Bronx, New York City']
G3 = ox.graph_from_place(places, clean_periphery=False, network_type='drive_service')
G3_projected = ox.project_graph(G3)
fig, ax = ox.plot_graph(G3_projected, fig_height=10, node_size=1)

### Method 4: Network from Place Polygon
This last method extracts all the streets within a polygon:

In [ ]:
# get the boundary polygon for Hong Kong and plot
# gdf is a geopandas GeoDataFrame
city = ox.gdf_from_place('Hong Kong, Hong Kong')
fig, ax = ox.plot_shape(city, figsize=(3,3))

In [ ]:
# now use the city polygon to extract streets from OSM
# the projection will take some time - can leave it running and do something else
city_polygon = city['geometry'].iloc[0]
G4 = ox.graph_from_polygon(city_polygon, network_type='walk')
G4_projected = ox.project_graph(G4)
fig, ax = ox.plot_graph(G4_projected)

One downside of the OSMnx tool is that it does not handle disconnected networks. In the example for network G4 above, if you specified a network_type='walk' for Hong Kong, you would only get Kowloon and the New Territories - Hong Kong Island and Lantau Island would be missing, because these networks are not connected to Kowloon as the tunnes and bridges interrupt the walking network. The solution is to use *Method 2* to download walking networks for *'Hong Kong, Hong Kong'*, *'Hong Kong Island, Hong Kong'*, and *'Lantau Island, Hong Kong'* separately and save them to shapefiles per the code below. 

## Saving your Networks to Shapefiles
OSMnx supports exporting your networks to shapefiles. Let's export network *G4* so we can use it in ArcGIS:

In [ ]:
# save the Hong Kong boundary polygon as a shapefile
ox.save_gdf_shapefile(city, folder='/data', filename='Hong_Kong')

# Save the Hong Kong street network as ESRI shapefile to work with in GIS
ox.save_graph_shapefile(G4, folder='/data', filename='G4_network_shape')

## Other Fun Things
### Figure-Ground Diagram
Use OSMnx to download square-mile city street networks and visualize them as figure-ground diagrams. Discussed in this [blog post](https://geoffboeing.com/2017/01/square-mile-street-network-visualization/)

In [ ]:
# configure the inline image display
from IPython.display import Image
img_folder = 'images'
extension = 'png'
size = 240
dpi = 40

Now let's try a few places:

In [ ]:
# Pass in the Latitude and Longitude of a location. Let's try TST
place = 'tst'
point = (22.3005, 114.172283)
fig, ax = ox.plot_figure_ground(point=point, filename=place, dpi=dpi)
Image('{}/{}.{}'.format(img_folder, place, extension), height=size, width=size)

In [ ]:
# Now Central
place = 'central'
point = (22.283905, 114.154418)
fig, ax = ox.plot_figure_ground(point=point, filename=place, dpi=dpi)
Image('{}/{}.{}'.format(img_folder, place, extension), height=size, width=size)

In [ ]:
# How about Times Square in NYC
place = 'nyc'
point = (40.759204, -73.984624)
fig, ax = ox.plot_figure_ground(point=point, filename=place, dpi=dpi)
Image('{}/{}.{}'.format(img_folder, place, extension), height=size, width=size)

### Running Network Statistics

In [ ]:
# define which network you would like to analyze
network = G1_projected

In [ ]:
# show some basic stats about the network
stats = ox.basic_stats(network)
stats
# see more info at https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.stats

### Visualize Nodes by Betweeness Centrality

In [ ]:
# run the extended statistics routine
extended_stats = ox.extended_stats(network, ecc=True, bc=True, cc=True)

# get a color for each node
def get_color_list(n, color_map='plasma', start=0, end=1):
    return [cm.get_cmap(color_map)(x) for x in np.linspace(start, end, n)]

def get_node_colors_by_stat(network, data, start=0, end=1):
    df = pd.DataFrame(data=pd.Series(data).sort_values(), columns=['value'])
    df['colors'] = get_color_list(len(df), start=start, end=end)
    df = df.reindex(network.nodes())
    return df['colors'].tolist()

nc = get_node_colors_by_stat(network, data=extended_stats['betweenness_centrality'])
fig, ax = ox.plot_graph(network, node_color=nc, node_edgecolor='gray', node_size=20, node_zorder=2)

### Visualize Streets by Closeness Centrality

In [ ]:
# edge closeness centrality: convert graph to line graph so edges become nodes and vice versa
edge_centrality = nx.closeness_centrality(nx.line_graph(network))

# list of edge values for the orginal graph
ev = [edge_centrality[edge + (0,)] for edge in network.edges()]

# color scale converted to list of colors for graph edges
norm = colors.Normalize(vmin=min(ev)*0.8, vmax=max(ev))
cmap = cm.ScalarMappable(norm=norm, cmap=cm.inferno)
ec = [cmap.to_rgba(cl) for cl in ev]

# color the edges in the original graph with closeness centralities in the line graph
fig, ax = ox.plot_graph(network, bgcolor='k', axis_off=True, node_size=0,
                        edge_color=ec, edge_linewidth=1.5, edge_alpha=1)

# References
Boeing, G. 2017. “OSMnx: New Methods for Acquiring, Constructing, Analyzing, and Visualizing Complex Street Networks.” *Computers, Environment and Urban Systems* 65, 126-139.